Importando as bibliotecas

In [2]:
import numpy as np
import pandas as pd
import sweetviz as sv
import os
import warnings
warnings.filterwarnings("ignore")

Garantindo que o diretório de trabalho está correto

In [3]:
base_dir = os.path.dirname(os.path.abspath('__file__'))
dir_files_path = os.path.join(base_dir, '../data/raw/2016.csv')
reports_file_path = os.path.join(base_dir, '../reports/2016')

Dataframes

In [14]:
licitacoes_2016 = pd.read_csv(f"{dir_files_path}/licitacao.csv")
licitantes_2016 = pd.read_csv(f"{dir_files_path}/licitante.csv")
itens_2016 = pd.read_csv(f"{dir_files_path}/item.csv")

Relatórios de análise exploratória

In [33]:
# licitacoes
report_licitacoes = sv.analyze(licitacoes_2016)
report_licitacoes.show_html(filepath=f"{reports_file_path}/licitacoes_report.html", layout='vertical')

                                             |          | [  0%]   00:00 -> (? left)

Report /home/vlgi/Área de Trabalho/meus_projetos/dsb/dsb/notebooks/../reports/2016/licitacoes_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [31]:
#itens
report_itens = sv.analyze(itens_2016)
report_itens.show_html(filepath=f"{reports_file_path}/itens_report.html", layout= 'vertical')

                                             |          | [  0%]   00:00 -> (? left)

TypeError: 

Column [VL_TOTAL_HOMOLOGADO] has a 'mixed' inferred_type (as determined by Pandas).
This is is not currently supported; column types should not contain mixed data.
e.g. only floats or strings, but not a combination.

POSSIBLE RESOLUTIONS:
BEST -> Make sure series [VL_TOTAL_HOMOLOGADO] only contains a certain type of data (numerical OR string).
OR -> Convert series [VL_TOTAL_HOMOLOGADO] to a string (if makes sense) so it will be picked up as CATEGORICAL or TEXT.
     One way to do this is:
     df['VL_TOTAL_HOMOLOGADO'] = df['VL_TOTAL_HOMOLOGADO'].astype(str)
OR -> Convert series [VL_TOTAL_HOMOLOGADO] to a numerical value (if makes sense):
     One way to do this is:
     df['VL_TOTAL_HOMOLOGADO'] = pd.to_numeric(df['VL_TOTAL_HOMOLOGADO'], errors='coerce')
     # (errors='coerce' will transform string values to NaN, that can then be replaced if desired; consult Pandas manual pages for more details)


O erro anterior aponta que na coluna VL_TOTAL_HOMOLOGADO consta uma combinação de tipos de variáveis - float e string
Logo, precisa-se explorar o dataframe para identificar o que está acontecendo

In [86]:
itens_2016.shape # (456368, 32)

(456368, 32)

In [87]:
itens_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456368 entries, 0 to 456367
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   CD_ORGAO                        456368 non-null  int64  
 1   NR_LICITACAO                    456368 non-null  float64
 2   ANO_LICITACAO                   456368 non-null  int64  
 3   CD_TIPO_MODALIDADE              456368 non-null  object 
 4   NR_LOTE                         456368 non-null  int64  
 5   NR_ITEM                         456368 non-null  int64  
 6   NR_ITEM_ORIGINAL                456368 non-null  object 
 7   DS_ITEM                         456368 non-null  object 
 8   QT_ITENS                        456368 non-null  float64
 9   SG_UNIDADE_MEDIDA               456368 non-null  object 
 10  VL_UNITARIO_ESTIMADO            447196 non-null  float64
 11  VL_TOTAL_ESTIMADO               447196 non-null  float64
 12  DT_REF_VALOR_EST

A coluna VL_TOTAL_HOMOLOGADO não consta no dicionário de dados apresentado e nem no site da transparência.
Irei tratar o dado para verificar o report, mas acredito que não seja um dado relevante para o estudo.

In [61]:
itens_2016[['QT_ITENS','VL_UNITARIO_HOMOLOGADO', 'VL_TOTAL_HOMOLOGADO']]

,QT_ITENS,VL_UNITARIO_HOMOLOGADO,VL_TOTAL_HOMOLOGADO
0,10.0,1430.000,14300.0
1,4.0,1258.000,5032.0
2,6.0,1998.000,11988.0
3,4.0,2978.000,11912.0
4,4.0,2500.000,10000.0
...,...,...,...
456363,10000.0,0.089,890.0
456364,300.0,7.100,2130.0
456365,100000.0,0.144,14400.0
456366,15000.0,0.620,9300.0


In [5]:
itens_2016[['VL_UNITARIO_HOMOLOGADO','VL_TOTAL_HOMOLOGADO']].isnull().sum()

VL_UNITARIO_HOMOLOGADO    83395
VL_TOTAL_HOMOLOGADO       83395
dtype: int64

In [6]:
372973 + 83395
# A soma dos valores nulos e não nulos é igual ao número de linhas do dataset

456368

Então, vou tratar os valores nulos e qualquer outro problema que possa existir para gerar o report

In [7]:
# Tratando possíveis problemas com os valores em reais
itens_2016['VL_TOTAL_HOMOLOGADO'] = itens_2016['VL_TOTAL_HOMOLOGADO'].str.replace(',','.')
itens_2016['VL_TOTAL_HOMOLOGADO'] = itens_2016['VL_TOTAL_HOMOLOGADO'].str.replace('R$','')

In [8]:
# Forçando a conversão para float
itens_2016['VL_TOTAL_HOMOLOGADO'] = pd.to_numeric(itens_2016['VL_TOTAL_HOMOLOGADO'], errors='coerce')

In [9]:
# Preenchendo os valores nulos com 0
itens_2016[['VL_UNITARIO_HOMOLOGADO','VL_TOTAL_HOMOLOGADO']] = itens_2016[['VL_UNITARIO_HOMOLOGADO','VL_TOTAL_HOMOLOGADO']].fillna(0)

In [10]:
# Checando se ainda há valores nulos
itens_2016[['VL_UNITARIO_HOMOLOGADO','VL_TOTAL_HOMOLOGADO']].isnull().sum()

VL_UNITARIO_HOMOLOGADO    0
VL_TOTAL_HOMOLOGADO       0
dtype: int64

Agora, consigo converter os valores para float

In [11]:
# this converts the values to float
itens_2016['VL_TOTAL_HOMOLOGADO'] = itens_2016['VL_TOTAL_HOMOLOGADO'].astype(float)

E, finalmente, gerar o report

In [12]:
#itens
report_itens = sv.analyze(itens_2016)
report_itens.show_html(filepath=f"{reports_file_path}/itens_report.html", layout= 'widescreen')

                                             |          | [  0%]   00:00 -> (? left)

TypeError: 

Column [NR_DOCUMENTO.1] has a 'mixed' inferred_type (as determined by Pandas).
This is is not currently supported; column types should not contain mixed data.
e.g. only floats or strings, but not a combination.

POSSIBLE RESOLUTIONS:
BEST -> Make sure series [NR_DOCUMENTO.1] only contains a certain type of data (numerical OR string).
OR -> Convert series [NR_DOCUMENTO.1] to a string (if makes sense) so it will be picked up as CATEGORICAL or TEXT.
     One way to do this is:
     df['NR_DOCUMENTO.1'] = df['NR_DOCUMENTO.1'].astype(str)
OR -> Convert series [NR_DOCUMENTO.1] to a numerical value (if makes sense):
     One way to do this is:
     df['NR_DOCUMENTO.1'] = pd.to_numeric(df['NR_DOCUMENTO.1'], errors='coerce')
     # (errors='coerce' will transform string values to NaN, that can then be replaced if desired; consult Pandas manual pages for more details)


Levando em conta que a coluna apresentada no erro anterior também não está no dicionário de dados, vou ignorá-la em conjunto com qualquer outra para gerar o report

In [94]:
#itens
report_itens = sv.analyze(itens_2016.drop(columns=['NR_DOCUMENTO.1']))
report_itens.show_html(filepath=f"{reports_file_path}/itens_report.html", layout= 'vertical')

                                             |          | [  0%]   00:00 -> (? left)

Report /home/vlgi/Área de Trabalho/meus_projetos/dsb/dsb/notebooks/../reports/2016/itens_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Por meio dos reports, é possível fazer um diagnóstico inicial do dataset e identificar possíveis respostas para as perguntas de negócio.

Agora, iremos começar pelas bases recomendadas e depois tentar relacionar com as demais com o objetivo de adquirir mais informações
e por fim responder às perguntas de negócio.

In [38]:
# licitacoes filtradas pelo tipo de objeto - compra
filtro = licitacoes_2016['TP_OBJETO'].str.match('com',case = False) & licitacoes_2016['DS_OBJETO'].str.contains('aquisição', case = False)
licitacoes_2016[filtro]['DS_OBJETO'].value_counts()

AQUISIÇÃO DE MEDICAMENTOS                                                                                                                                                                                                                                                                                            36
Aquisição de medicamentos                                                                                                                                                                                                                                                                                            24
Aquisição de material hospitalar                                                                                                                                                                                                                                                                                     23
Aquisição de Medicamentos                                       

In [17]:
itens_2016

,CD_ORGAO,NR_LICITACAO,ANO_LICITACAO,CD_TIPO_MODALIDADE,NR_LOTE,NR_ITEM,NR_ITEM_ORIGINAL,DS_ITEM,QT_ITENS,SG_UNIDADE_MEDIDA,...,TP_ORCAMENTO,CD_TIPO_FAMILIA,CD_TIPO_SUBFAMILIA,TP_DOCUMENTO,NR_DOCUMENTO,TP_DOCUMENTO.1,NR_DOCUMENTO.1,TP_BENEFICIO_MICRO_EPP,PC_TX_ESTIMADA,PC_TX_HOMOLOGADA
0,77200,15.0,2016,PRP,1,2,2,"Pneu 1000/20 liso radial, dianteiro, 16 lonas,...",10.0,UN,...,M,745.0,632.0,J,8.774832e+12,NaN,NaN,L,NaN,NaN
1,77200,15.0,2016,PRP,1,4,4,"Pneu 12.4x24 garra baixa, 10 lonas",4.0,UN,...,M,745.0,632.0,J,8.774832e+12,NaN,NaN,L,NaN,NaN
2,77200,15.0,2016,PRP,1,7,7,Pneu 1400x24 mínimo 12 lonas,6.0,UN,...,M,745.0,632.0,J,8.774832e+12,NaN,NaN,L,NaN,NaN
3,77200,15.0,2016,PRP,1,8,8,"Pneu 17.5x25 12 lonas, capacidade mínima de ca...",4.0,UN,...,M,745.0,632.0,J,8.774832e+12,NaN,NaN,L,NaN,NaN
4,77200,15.0,2016,PRP,1,12,12,"Pneu 18.4x30 garra baixa, mínimo 10 lonas",4.0,UN,...,M,745.0,632.0,J,8.774832e+12,NaN,NaN,L,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456363,78900,13.0,2016,PRP,1,154,154,Ranitidina 150mg,10000.0,COM,...,NaN,NaN,NaN,J,5.782733e+12,NaN,NaN,N,NaN,NaN
456364,78900,13.0,2016,PRP,1,156,156,Salbutamol 100mcg/dose aerosol 200 doses,300.0,FR,...,NaN,NaN,NaN,J,5.782733e+12,NaN,NaN,N,NaN,NaN
456365,78900,13.0,2016,PRP,1,158,158,Sinvastatina 40mg,100000.0,COM,...,NaN,NaN,NaN,J,5.782733e+12,NaN,NaN,N,NaN,NaN
456366,78900,13.0,2016,PRP,1,167,167,Valproato de Sódio 500 mg,15000.0,COM,...,NaN,NaN,NaN,J,5.782733e+12,NaN,NaN,N,NaN,NaN
